# Map Function

In [ ]:


import tensorflow as tf
tf.reset_default_graph();

batch_size=32;
n_entities = 1000;
primus = tf.random_normal([batch_size,200])
elems = tf.random_normal([n_entities,200])
def f(x):
    x_ = tf.tile(tf.expand_dims(x,axis=0),[batch_size,1]);
    return tf.reduce_sum(tf.pow(primus-x_,2),axis=-1);
y = tf.transpose(tf.map_fn(f, elems))
z = tf.reduce_sum(tf.pow(tf.tile(tf.expand_dims(primus,1),[1,n_entities,1])-tf.tile(tf.expand_dims(elems,0),[batch_size,1,1]),2),axis=-1);
with tf.Session(graph=tf.get_default_graph()) as sess:
    sess.run(tf.global_variables_initializer());
    yy,zz = sess.run([y,z])

# Multiple Graphs

In [ ]:
'''
Reading:
1. https://stackoverflow.com/questions/35955144/working-with-multiple-graphs-in-tensorflow
2. https://stackoverflow.com/questions/39614938/why-do-we-need-tensorflow-tf-graph

TensorFlow:
1. https://www.tensorflow.org/api_docs/python/tf/initializers
2. https://www.tensorflow.org/api_docs/python/tf/get_variable
3. https://www.tensorflow.org/api_docs/python/tf/GraphKeys
4. https://www.tensorflow.org/programmers_guide/saved_model
5. https://www.tensorflow.org/api_docs/python/tf/variable_scope
'''

In [ ]:
import tensorflow as tf
import numpy as np

tf.reset_default_graph(); # Clean the slate
g = tf.get_default_graph(); # The default tf graph

# Graph-1
tf.reset_default_graph(); # Clean the slate
g1 = tf.Graph();
with g1.as_default():
    with tf.variable_scope('g1', reuse=tf.AUTO_REUSE):
        inp_g1 = tf.placeholder(tf.float32, shape=[5,2], name='inp_g1');
        var_g1 = tf.get_variable("var_g1", shape=[2,5], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        prod_g1 = tf.matmul(inp_g1, var_g1, name='prod_g1');
        print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES));
# Graph-2
tf.reset_default_graph(); # Clean the slate
g2 = tf.Graph();
with g2.as_default():
    with tf.variable_scope('g2'):
        inp_g2 = tf.placeholder(tf.float32, shape=[5,2], name='inp_g2');
        var_g2 = tf.get_variable("var_g2", shape=[5,2], initializer=tf.random_uniform_initializer(minval=-5, maxval=5))
        prod_g2 = tf.multiply(inp_g2, var_g2, name='prod_g2');
        print(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES));

# Data from external source
inp_ = np.random.randint(low=-1,high=2,size=10).reshape([5,2])

# Run something g1 and save
tf.reset_default_graph(); # Clean the slate
with tf.Session(graph=g1) as sess:
    sess.run(tf.global_variables_initializer());
    prod_g1_ = sess.run([prod_g1],feed_dict={inp_g1:inp_})[0];
    saver = tf.train.Saver();
    saver.save(sess, './g1_session.ckpt');
# restore the graph and get selected variable's data
tf.reset_default_graph(); # Clean the slate
with tf.Session(graph=g1) as sess: # Specifying graph=g1 is a MUST here
    saver = tf.train.Saver();
    saver.restore(sess, './g1_session.ckpt')
    var_g1__ = g1.get_tensor_by_name('g1/var_g1:0');
    print('printing... ', var_g1);
    var_g1_ = sess.run([var_g1__])[0];